In [120]:
import pandas as pd
import numpy as np
import GPy
from collections import defaultdict

import seaborn as sns

%matplotlib inline

In [121]:
cd ../../../data/helpers/skills

/Users/Logan/Google Drive/Oxford/DPhil/future_employment/data/helpers/skills


In [122]:
X = pd.read_csv('skills_2009.csv')
Y = pd.read_csv('automation_targets.csv')
codes = pd.read_csv('codes_index.csv')
y = pd.read_csv('automation_y.csv')

In [123]:
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA

## UNSOLVED
# Using time series data – multiple years

## NOTES ABOUT GP MODELS
# optimize hyperparameters
# perform random restarts
# allow for multiple kernels
# optimize kernels
# optimize variances

## TO DO
# percentile regression model evaluation

# CREATE REGRESSION MODELS
from GPy.models import GPRegression
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from GPy.models import GPClassification
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

In [124]:
reg_models = [('gpreg', GPRegression),
			  ('bayes_ridge', BayesianRidge()),
			  ('Gboost', GradientBoostingRegressor()),
			  ('support_vec_reg', SVR())
			 ]

# CREATE CLASSIFICATION MODELS
class_models = [('gpclass', GPClassification),
				('ridge_class', RidgeClassifier()),
				('gbclass', GradientBoostingClassifier()),
				('support_vec_class', SVC()),
				('naivebayes', BernoulliNB())
		 		]

In [125]:
# CREATE INPUTS LIST
def get_array_percentiles(array):
		def percentile(x, array):
			return 100*np.mean(array <= x)

		return np.array(map(lambda x: percentile(x, array), array))

def get_percentiles(df):
	if isinstance(df, pd.DataFrame):
		new_df = df.copy()
		return new_df.apply(lambda x: get_array_percentiles(x), axis = 0)
	elif isinstance(df, np.ndarray):
		return np.apply_along_axis(get_percentiles, 0, a)
	else:
		print "TYPE ERROR; PLEASE INPUT pd.DataFrame OR np.ndarray"
		raise

In [126]:
X = pd.read_csv("skills_2009.csv")
X_skills, X_task, X_context = X, X, X
# X_skills = pd.read_csv("X_skills.csv")
# X_task = pd.read_csv("X_task.csv")
# X_context = pd.read_csv("X_context.csv")

In [127]:
Y = pd.read_csv("automation_targets.csv")
Y.columns = ["O*NET-SOC Code", "auto_15", "auto_9", "delta", "auto_delta_pct", "title"]

In [128]:
codes = pd.read_csv("codes_index.csv")
full_X = pd.concat((X_skills, codes), axis = 1)

In [129]:
full_X = full_X.merge(Y, on = "O*NET-SOC Code")

In [130]:
inputs = []
for dX in [full_X]:
    pctiles = get_percentiles(dX)
    d_list =  [('dx', dX), ('pctile', pctiles)]
    for d in d_list:
        prefix = d[0]
        data = d[1]
        inputs.append((prefix, data.iloc[:,:-6]))
        inputs.append((prefix + '_greater_than', data[data.auto_delta_pct > 0].iloc[:,:-6]))
        inputs.append((prefix + '_less_than', data[data.auto_delta_pct < 0].iloc[:,:-6]))
# 		inputs.append(data[emp_delta_pct > 0])
# 		inputs.append(data[emp_delta_pct < 0])

# new_inputs = []
# for inp in inputs:
#     new_inputs.append(inp[0], inp[1].iloc[:,:-6])

# inputs = new_inputs

In [132]:
# FINAL DATA
y_out = full_X[['auto_9', 'auto_15', 'delta', 'auto_delta_pct']]
codes = full_X["O*NET-SOC Code"]

full_X = full_X.drop(['O*NET-SOC Code', 'auto_9', 'auto_15', 'delta', 'auto_delta_pct', 'title'], axis = 1)

In [16]:
# automation = pd.read_csv("y_automation.csv")
# computerisation = pd.read_csv("y_computerisation.csv")

In [ ]:
get_

In [145]:
# CREATE REG TARGETS
y_before = ('before', y_out.auto_9)
y_after = ('after', y_out.auto_15)
y_delta = ('delta', y_out.delta)
y_pct = ('delta_pct', y_out.auto_delta_pct)
reg_targets = [y_before, y_after, y_delta, y_pct]

new_reg_targets = []
for name, series in reg_targets:
    new_reg_targets.append((name, series))
    new_name = name + '_pctile'
    new_series = get_array_percentiles(series)
    new_reg_targets.append((new_name, new_series))
    
reg_targets = new_reg_targets

In [146]:
# CREATE CLASS TARGETS
class_targets = []
pos_threshold_values = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00]
neg_threshold_values = map(lambda x: -x, pos_threshold_values)
threshold_values = pos_threshold_values + neg_threshold_values

for name, target in reg_targets:
    for sd_threshold_value in threshold_values:
        low_threshold = target > (np.mean(target) - sd_threshold_value*np.std(target))
        new_name = str(sd_threshold_value)+"_gt_" + name
        class_targets.append((new_name,low_threshold))

# 	mean_threshold = target > np.mean(target)
# 	new_name = 'mean_gt_' + name
# 	class_targets.append((name, mean_threshold))

In [ ]:
X_inputs = inputs
y_inputs_reg, y_inputs_class = reg_targets, class_targets
models_reg, models_class = reg_models, class_models

In [ ]:
X = [X_inputs[0]]
y = [y_inputs_reg[4]]
m = [('GP REG', GPRegression)]

In [ ]:
X = np.array(X_inputs[0][1])
y = np.array(y_inputs_reg[4][1])[:,np.newaxis]

In [ ]:
X.shape, y.shape

In [ ]:
preds = m.predict(te_x)

In [ ]:
tr_x, te_x, tr_y, te_y = train_test_split(X, y)
m = GPRegression(tr_x, tr_y)
m.optimize()
m.optimize_restarts(5)


In [ ]:
score_model(preds, te_y, ('gpreg', X), m, 'regression')

In [ ]:
#### aggregate models as list of tuples [(name, val), (name, val)]
# loop

def run_loop(X_inputs, y_inputs_reg = None, y_inputs_class = None, models_reg = None, models_class = None):
    ## returns all models in comparison, as dict
    tree = lambda: defaultdict(tree)
    evals = tree()
    
    # begin loop over inputs
    for X_inp in X_inputs: #FOR ALL INPUTS
        X_name, X = X_inputs[0], X_inputs[1]
        
        # check if null pred_type
        if y_inputs_reg and not y_inputs_class:
            pred_type = 'regression'
        
        elif y_inputs_class and not y_inputs_reg:
            pred_type = 'classification'
        
        elif not(y_inputs_reg or y_inputs_class):
            raise "Input a target variable!"
        
        else: # both are fulfilled; perform classification & regression
            new_dict = evals.copy()
            for pred_type in ['regression', 'classification']:
                if pred_type == 'regression':
                    y_inps = y_inputs_reg
                    mods = models_reg
                elif pred_type == 'classification':
                    y_inputs = y_inputs_class
                    mods = models_class
                else:
                    return "SOMETHING WRONG!"
                new_dict.update(run_model())
                evals = new_dict
        
        evals = run_model()
                
        def run_model(pred_type):
            model_evals = tree()
        
            # do inference over all targets

            for y_inp in y_inps:
                # format outcome data
                y_name, y = y_inp[0], y_inp[1]
                if y.ndim == 1:
                    y = y[:, np.newaxis]

                #format feature matrix data
                X_train, X_test, y_train, y_test = train_test_split(X, y)
                data = [X_train, X_test, y_train, y_test]
                new_data = []
                # check that all data is np.ndarry
                for d in data:
                    if not isinstance(data, np.ndarray):
                        new_data.append(np.array(data))
                    else:
                        new_data.append(data)
                X_train, X_test, y_train, y_test = new_data

                #loop over models
                for mod in mods:
                    model_name, model = models[0], models[1]
                    try:
                        model.fit(X_train, y_train)
                    except AttributeError:
                        model = model(X_train, y_train) # THESE ARE GPs!
                        model.optimize()
                        model.optimize_restarts(20)
                    y_pred = model.predict(X_test)
                    score = score_model(y_pred, y_test, X_inp, mod, score_type = pred_type) # CREATE TYPES
                    model_evals[y_name][model_name][X_name] = score
                return model_evals

        return evals


In [ ]:
def score_model(y_pred, y_test, X_inputs, models, score_type):
    score = {}
    X_name, X = X_inputs[0], X_inputs[1]
    model_name, model = models[0], models[1]

    if score_type == "classification":
        accuracy = -1
        precision = None
        recall = None
        specificity = None
        f1 = None
        ideal_cutoff = None

        chart_storage = np.array([None, None, None])
        for cutoff in np.arange(0, 1.001, 0.001):
            y_pred = y_pred > cutoff
            num_TP = float(np.sum((y_pred == 1) & (y_test == 1)))
            num_FP = float(np.sum((y_pred == 1) & (y_test == 0)))
            num_TN = float(np.sum((y_pred == 0) & (y_test == 0)))
            num_FN = float(np.sum((y_pred == 0) & (y_test == 1)))

            s_accuracy = np.mean(y_pred == y_test)
            s_precision = num_TP / (num_TP + num_FP)
            s_recall = num_TP / (num_TP + num_FN)
            s_specificity = num_TN / (num_TN + num_FP)
            s_f1 = (precision * recall) / (precision + recall)

            chart_storage.append([cutoff, s_recall, s_specificity])

            if s_accuracy > accuracy:
                ideal_cutoff = cutoff
                precision = s_precision
                recall = s_recall
                specificity = s_specificity 
                f1 = s_f1
                ideal_cutoff = s_ideal_cutoff

        def plot_chart(chart_storage):
            cutoffs = chart_storage[:,0]
            sensitivity = chart_storage[:,1]
            specificity = chart_storage[:,2]
            sns.plt.plot(sensitivity, specificity)
            sns.plt.close()

        eval_text = """
        Model 		| 	{}		|
        X name		|	{}		|
        Accuracy	|	{}		|
        Precision  	|	{}		|
        Recall 		|	{}		|
        Specificity |	{}		|
        F1			|	{}		|
        """.format(model_name,
                   X_name,
                   accuracy,
                   precision,
                   recalll,
                   specificity,
                   f1)
        score['Model name'] = model_name
        score['Model'] = model
        score['X_name'] = X_name
        score['Accuracy'] = accuracy
        score['Precision'] = precision
        score['Recall'] = recall
        score['Specificity'] = specificity
        score['F1'] = F1
        score['AUC_data'] = chart_storage
        score['chart'] = plot_chart # when called, call on plot_chart( score['AUC_data'])

        print eval_text
        return score

    elif score_type == 'regression':
        errors = y_test - y_pred
        se = float(np.std(errors)) / np.sqrt(len(errors))

        eval_text = """
        Model 		| 	{}		|
        X name		|	{}		|
        Mean Y		|	{}		|
        Mean Y_hat	|	{}		|
        Mean error	|	{}		|
        SE			| 	{}		|
        Likelihood	|	##		|
        """.format(model_name,
                   X_name,
                   np.mean(y_test),
                   np.mean(y_pred),
                   np.mean(errors),
                   se
#                    likelihood
                   )

        ## plot pca

        ## plot errors

        print eval_text
        return score
    else:
        return "Please input a valid score_type {'regression', 'classification'}"

In [ ]:
def do_ARD(model, feature_names):
	# uses ARD to find relevant features
	length_scales = model.length_scales # np.ndarray
	features = range(1, len(model.features) + 1)
	importances = 1./length_scales
	cutoff = 0.25 * min(importances) # note: arbitrary
	important_features = features * (importances >= cutoff)
	if_indices = np.trim_zeros(important_features)
	return zip(feature_names[if_indices], importances[if_indices])